# **Import Required Libraries**

In [1]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# Install necessary libraries
!pip install transformers accelerate opencv-python matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# **Downloading Sapiens Depth and Pose Models from Hugging Face**

In [3]:
!wget https://huggingface.co/facebook/sapiens-depth-1b-torchscript/resolve/main/sapiens_1b_render_people_epoch_88_torchscript.pt2 -O sapiens-depth-1b.pt

--2025-02-12 10:43:32--  https://huggingface.co/facebook/sapiens-depth-1b-torchscript/resolve/main/sapiens_1b_render_people_epoch_88_torchscript.pt2
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.4, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a7/10/a710e69e3dd1d4c10d5b4b2e132ca47b78cfb2dd0e63648ceabc95ab8d4fc5f3/4ff0c7a8fa48f1d30f97a49aee05abb905f64ee4fe6a35efa805821be5756a8c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sapiens_1b_render_people_epoch_88_torchscript.pt2%3B+filename%3D%22sapiens_1b_render_people_epoch_88_torchscript.pt2%22%3B&Expires=1739360612&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTM2MDYxMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2E3LzEwL2E3MTBlNjllM2RkMWQ0YzEwZDViNGIyZTEzMmNhNDdiNzhjZmIyZGQwZTYzNjQ4Y2VhYmM5N

# **Model Initilization**




In [4]:
!wget https://huggingface.co/facebook/sapiens-pose-1b-torchscript/resolve/main/sapiens_1b_goliath_best_goliath_AP_639_torchscript.pt2 -O sapiens-pose-1b.pt


--2025-02-12 10:45:59--  https://huggingface.co/facebook/sapiens-pose-1b-torchscript/resolve/main/sapiens_1b_goliath_best_goliath_AP_639_torchscript.pt2
Resolving huggingface.co (huggingface.co)... 108.156.91.74, 108.156.91.122, 108.156.91.37, ...
Connecting to huggingface.co (huggingface.co)|108.156.91.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/58/78/5878ffb4f3c9a6e45887705f398a57f2b5e78cdb65e7687bfd112112c53de0f9/d1077c15e66fb2798893e569f71cb063f0b0778af2a62e3c0233807a351c31f3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sapiens_1b_goliath_best_goliath_AP_639_torchscript.pt2%3B+filename%3D%22sapiens_1b_goliath_best_goliath_AP_639_torchscript.pt2%22%3B&Expires=1739360759&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTM2MDc1OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzU4Lzc4LzU4NzhmZmI0ZjNjOWE2ZTQ1ODg3NzA1ZjM5OGE1N2YyYjVlNzhjZGI2NWU

In [5]:
import torch
from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation

# Load Sapiens Pose Model
pose_model = torch.jit.load('/content/sapiens-pose-1b.pt')
pose_model.eval()

# Load Sapiens Depth Model
depth_model = torch.jit.load('/content/sapiens-depth-1b.pt')
depth_model.eval()

RecursiveScriptModule(
  original_name=DepthEstimator
  (data_preprocessor): RecursiveScriptModule(original_name=SegDataPreProcessor)
  (backbone): RecursiveScriptModule(
    original_name=VisionTransformer
    (patch_embed): RecursiveScriptModule(
      original_name=PatchEmbed
      (adaptive_padding): RecursiveScriptModule(original_name=AdaptivePadding)
      (projection): RecursiveScriptModule(original_name=Conv2d)
    )
    (drop_after_pos): RecursiveScriptModule(original_name=Dropout)
    (layers): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=TransformerEncoderLayer
        (ln1): RecursiveScriptModule(original_name=LayerNorm)
        (attn): RecursiveScriptModule(
          original_name=MultiheadAttention
          (qkv): RecursiveScriptModule(original_name=Linear)
          (proj): RecursiveScriptModule(original_name=Linear)
          (proj_drop): RecursiveScriptModule(original_name=Dropout)
          (out_drop):

In [6]:
video_path = '/content/Kirolos_video.mp4'
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Select start frame, a couple of middle frames, and the last frame
selected_indices = [0, total_frames // 4, total_frames // 2, (3 * total_frames) // 4, total_frames - 1]
frames = []

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx in selected_indices:
        frames.append(frame)
    frame_idx += 1

cap.release()


In [7]:
def process_frame_with_sapiens_pose(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image, (1024, 768))
    image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).unsqueeze(0).float() / 255.0

    with torch.no_grad():
        pose_output = pose_model(image_tensor)

    # Assuming pose model returns landmarks in a structured way, adjust this based on actual model output
    landmarks = pose_output[0].cpu().numpy()  # (num_landmarks, 2)
    return landmarks

In [8]:
def process_frame_with_sapiens_depth(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image, (1024, 768))
    image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).unsqueeze(0).float() / 255.0

    with torch.no_grad():
        depth_map = depth_model(image_tensor)[0, 0].cpu().numpy()

    return depth_map

In [9]:
def calculate_3d_distance(point1, point2):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2)) ** 2))

In [ ]:
for idx, frame in enumerate(frames):
    landmarks = process_frame_with_sapiens_pose(frame)
    depth_map = process_frame_with_sapiens_depth(frame)

    left_ankle = landmarks[27]
    right_ankle = landmarks[28]

    # Get depth values for left and right ankles
    left_ankle_depth = depth_map[int(left_ankle[1]), int(left_ankle[0])]
    right_ankle_depth = depth_map[int(right_ankle[1]), int(right_ankle[0])]

    # Construct 3D points
    left_ankle_3d = (left_ankle[0], left_ankle[1], left_ankle_depth)
    right_ankle_3d = (right_ankle[0], right_ankle[1], right_ankle_depth)

    # Calculate distance between feet
    distance_between_feet = calculate_3d_distance(left_ankle_3d, right_ankle_3d)

    print(f"Frame {selected_indices[idx]}: Distance between feet = {distance_between_feet:.2f} units")

    # Optional: Display the frame with landmarks
    frame_with_landmarks = cv2.circle(frame, (int(left_ankle[0]), int(left_ankle[1])), 5, (0, 255, 0), -1)
    frame_with_landmarks = cv2.circle(frame_with_landmarks, (int(right_ankle[0]), int(right_ankle[1])), 5, (255, 0, 0), -1)

    plt.imshow(cv2.cvtColor(frame_with_landmarks, cv2.COLOR_BGR2RGB))
    plt.title(f"Pose Estimation - Frame {selected_indices[idx]}")
    plt.axis('off')
    plt.show()